In [ ]:
import pandas as pd

df_jan = pd.read_csv('../../data/bicing/processed/months/2023_01_STATIONS.csv')
df_feb = pd.read_csv('../../data/bicing/processed/months/2023_02_STATIONS.csv')
df_march = pd.read_csv('../../data/bicing/processed/months/2023_03_STATIONS.csv')
df_april = pd.read_csv('../../data/bicing/processed/months/2023_04_STATIONS.csv')
df_may = pd.read_csv('../../data/bicing/processed/months/2023_05_STATIONS.csv')
df_june = pd.read_csv('../../data/bicing/processed/months/2023_06_STATIONS.csv')
df_july = pd.read_csv('../../data/bicing/processed/months/2023_07_STATIONS.csv')
df_aug = pd.read_csv('../../data/bicing/processed/months/2023_08_STATIONS.csv')
df_sep = pd.read_csv('../../data/bicing/processed/months/2023_09_STATIONS.csv')
df_oct = pd.read_csv('../../data/bicing/processed/months/2023_10_STATIONS.csv')
df_nov = pd.read_csv('../../data/bicing/processed/months/2023_11_STATIONS.csv')
df_dec = pd.read_csv('../../data/bicing/processed/months/2023_12_STATIONS.csv')


In [ ]:
df_jan = df_jan[(df_jan['month'] == 1) & (df_jan['year'] == 2023)]
df_feb = df_feb[(df_feb['month'] == 2) & (df_feb['year'] == 2023)]
df_march = df_march[(df_march['month'] == 3) & (df_march['year'] == 2023)]
df_april = df_april[(df_april['month'] == 4) & (df_april['year'] == 2023)]
df_may = df_may[(df_may['month'] == 5) & (df_may['year'] == 2023)]
df_june = df_june[(df_june['month'] == 6) & (df_june['year'] == 2023)]
df_july = df_july[(df_july['month'] == 7) & (df_july['year'] == 2023)]
df_aug = df_aug[(df_aug['month'] == 8) & (df_aug['year'] == 2023)]
df_sep = df_sep[(df_sep['month'] == 9) & (df_sep['year'] == 2023)]
df_oct = df_oct[(df_oct['month'] == 10) & (df_oct['year'] == 2023)]
df_nov = df_nov[(df_nov['month'] == 11) & (df_nov['year'] == 2023)]
df_dec = df_dec[(df_dec['month'] == 12) & (df_dec['year'] == 2023)]

In [ ]:
df_2023 = pd.concat([df_jan, df_feb, df_march, df_april, df_may, df_june, df_july, df_aug, df_sep, df_oct, df_nov, df_dec])
df_2023.sort_values(by=['grouped_date'], inplace=True)
df_2023.head()

In [ ]:
df_2023['season'] = df_2023['grouped_date'].dt.month.apply(lambda x: 'Spring' if x in [3, 4, 5] else 'Summer' if x in [6, 7, 8] else 'Autumn' if x in [9, 10, 11] else 'Winter')
df_2023.head(4)

In [ ]:
df_weather = pd.read_csv('../../data/weather_clean.csv')

df_weather.head(3)

In [ ]:
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather['day'] = df_weather['date'].dt.day.astype(int)
df_weather['month'] = df_weather['date'].dt.month.astype(int)
df_weather['year'] = df_weather['date'].dt.year.astype(int)
df_weather['hour'] = df_weather['hour'].astype(int)

In [ ]:
df_2023_weather = pd.merge(df_2023, df_weather, on=['year', 'month', 'day', 'hour'], how='left')
df_2023_weather.head(3)

In [ ]:
station_info_df = pd.read_csv('../../data/bicing/processed/2024_STATION_LOCATIONS.csv', dtype={'post_code': str})
station_info_df.head()

In [ ]:
df_2023_weather_station = pd.merge(df_2023_weather, station_info_df, on='station_id', how='left')
df_2023_weather_station.head()

In [ ]:
df_2023_weather_station.columns

In [ ]:
columns_to_keep = ['station_id', 'num_bikes_available',
                   'num_bikes_available_types.mechanical',
                   'num_bikes_available_types.ebike', 'num_docks_available', 'status',
                   'is_renting', 'is_returning', 'day_of_week', 'is_weekend', 'grouped_date',
                   'docking_available', 'bikes_available', 'bikes_available.mechanical',
                   'bikes_available.ebike', 'temperature', 'rainfall', 'lat', 'lon', 'post_code', 'altitude', 'capacity', 'year', 'month', 'day', 'hour','season'
       'grouped_minute']

df_2023_ml = df_2023_weather_station[columns_to_keep]
df_2023_ml.head()

In [ ]:
df_2023_ml[df_2023_ml['capacity'].isnull()]['station_id'].unique()
#Station 520 missing information
df_2023_ml = df_2023_ml[df_2023_ml['station_id'] != 520]

In [ ]:
df_2023_ml.isnull().sum()

In [ ]:
df_2023_ml['grouped_date'] = pd.to_datetime(df_2023_ml['grouped_date'])

In [ ]:
holidays = ['01/06/2023', '04/07/2023', '04/10/2023', '05/01/2023', '06/24/2023', '08/15/2023', '09/11/2023', '10/12/2023', '11/01/2023', '12/06/2023', '12/08/2023', '12/25/2023', '12/26/2023']
df_2023_ml['is_holiday'] = df_2023_ml['grouped_date'].isin(holidays)
df_2023_ml[df_2023_ml['is_holiday'] == True]['grouped_date']

In [ ]:
df_2023_ml['percentage_docks_available'] = df_2023_ml['num_docks_available'] / df_2023_ml['capacity']
df_2023_ml.head(5)

In [ ]:
#filter stations if needed
stations_ids = df_2023_ml['station_id'].unique()
# get 5 random stations
import random
filter_stations = random.sample(list(stations_ids), 5)
#filter_stations = [57, 446]
filter_stations = [57, 32, 31, 378, 446]
filter_stations = [57]
df_a = df_2023_ml[df_2023_ml['station_id'].isin(filter_stations)]
#df_a = df_2023_ml

## LightGBM - Binary Target

### Pipeline

In [ ]:
import lightgbm as lgb

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer



df_lgbm = df_a.copy()

# Select features and target
features = ['day_of_week', 'hour', 'grouped_minute', 'temperature', 'rainfall',
            'capacity',  'season', 'altitude', 'station_id', 'is_weekend', 'month', 'day']
# removed month and day
# removed feautures that are not important disctrict, is_holiday
target = 'docking_available'

X = df_lgbm[features]
y = df_lgbm[target]

# I could remove day
categorical_features = ['day_of_week',  'season', 'station_id',
                        'is_weekend', 'hour', 'grouped_minute', 'month']
numerical_features = ['temperature', 'rainfall', 'capacity', 'altitude']


# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model_lgb = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', n_jobs=-1, learning_rate=0.05, num_leaves=31)


# Create pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model_lgb)])

pipeline

### TRain

In [ ]:
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
start_time = time.time()
pipeline.fit(X_train, y_train)
elapsed_time = time.time() - start_time

# Predictions
y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f"Training Time: {elapsed_time} seconds")
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)
print(f"ROC AUC Score: {roc_auc}")



## FEature Importance

In [ ]:
# Feature Importance
# Extract the model from the pipeline
model_rf = pipeline.named_steps['model']
importances = model_rf.feature_importances_

# Get feature names after preprocessing
encoded_categorical_features = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = numerical_features + list(encoded_categorical_features)

# Create DataFrame for feature importances
feature_importances = pd.DataFrame({'feature': all_feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importance")
plt.barh(range(len(feature_importances)), feature_importances['importance'], align="center")
plt.yticks(range(len(feature_importances)), feature_importances['feature'])
plt.gca().invert_yaxis()
plt.xlabel("Importance")
plt.ylabel("Features")
plt.show()